In [1]:
import os

## 1. 選擇使用的GPU 

In [2]:
# manually select one or several free gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
# use CPU only
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## 2. 到當前資料夾
- cd C:/Users/Garin/Desktop/學長畢業資料/實驗程式yu/

## 3. 同步程式
### - local -> server
- scp -r -P 2222 ./config garin@140.113.214.40:/home/garin/Documents/depth/
- scp -r -P 2222 ./src garin@140.113.214.40:/home/garin/Documents/depth/

### - server -> local 
- scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/config .
- scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/src .

## 4. 建dataset

In [ ]:
%run data_preparation/kitti_data_prepare.py \
--dataset_dir=/home/mjchiu/Documents/darknet-depth/dataset/KITTI/image/ \
--dataset_name=kitti_raw_eigen \
--dump_root=/home/garin/Documents/depth/datasets/kitti_3frames_256_832/ \
--seq_length=3 \
--img_height=256 \
--img_width=832 \
--num_threads=16 \
--remove_static

## 5. Train flow 

In [ ]:
%run ./main.py -c ../config/flow3.ini -t train_flow --cont_model=../results/KITTI_RAW_128_416_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-170987

## 6. Train depth & pose 

In [ ]:
%run ./main.py -c ../config/dp3.ini -t train_dp \
--restore_flow_model=../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-392302

#### continue training depth and pose 

In [ ]:
%run ./main.py -c ../config/dp3.ini -t train_dp \
--cont_model=../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_all_3frames/checkpoints/kitti_3frames/model-100591 \
--restore_flow_model=../results/KITTI_RAW_256_832_UnDepthflow_flow_pwc_b8_3frames/checkpoints/kitti_3frames/model-392302

[!] Loading the model for 3 frames...
[Info] Building depth and pose network ...
[Info] img_height: 256 img_width 832
[Info] Building depth and pose network ...
[Info] img_height: 256 img_width 832
[Info] Model size: 6.42670M
[Info] Continue training. Restoreing: ../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_all_3frames/checkpoints/kitti_3frames/model-100591
INFO:tensorflow:Restoring parameters from ../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_all_3frames/checkpoints/kitti_3frames/model-100591


INFO:tensorflow:Restoring parameters from ../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_all_3frames/checkpoints/kitti_3frames/model-100591


Epoch: [11] [   11/10059] total steps:[100601] lr:[0.00005000] time: 633.80s (633s total) loss: 170.927
Epoch: [11] [   21/10059] total steps:[100611] lr:[0.00005000] time: 225.72s (859s total) loss: 112.482
Epoch: [11] [   31/10059] total steps:[100621] lr:[0.00005000] time: 16.91s (876s total) loss: 107.570
Epoch: [11] [   41/10059] total steps:[100631] lr:[0.00005000] time: 15.16s (891s total) loss: 129.715
Epoch: [11] [   51/10059] total steps:[100641] lr:[0.00005000] time: 15.22s (906s total) loss: 184.795
Epoch: [11] [   61/10059] total steps:[100651] lr:[0.00005000] time: 15.10s (921s total) loss: 133.360
Epoch: [11] [   71/10059] total steps:[100661] lr:[0.00005000] time: 14.97s (936s total) loss: 92.018
Epoch: [11] [   81/10059] total steps:[100671] lr:[0.00005000] time: 14.67s (951s total) loss: 172.264
Epoch: [11] [   91/10059] total steps:[100681] lr:[0.00005000] time: 14.89s (966s total) loss: 108.800
Epoch: [11] [  101/10059] total steps:[100691] lr:[0.00005000] time: 15.

## 7. Test

In [3]:
%run ./main.py -c ../config/test_dp_kitti.ini -t kitti_eval \
--restore_dp_model=../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_all_3frames/checkpoints/kitti_3frames/model-100591
# --restore_dp_model=../results/KITTI_RAW_128_416_UnDepthflow_dp_b4_resnet50_3frames/checkpoints/kitti_3frames/model-342007


[Info] Evaluate kitti depth...
[Info] Reading datalist from: /home/garin/Documents/depth/src/kitti_eval/kitti/test_files_eigen.txt
[Info] Loading images from: /home/waterman/dataset/KITTI
[Info] Reshaing image to size: (256, 832)
[Info] Restoring model: ../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_all_3frames/checkpoints/kitti_3frames/model-100591
[Info] Data number: 697
[Info] FPS: 25.820
[Info] Saving to ../results/kitti/test_kitti.npy


In [4]:
%run kitti_eval/eval_depth.py --split=eigen --kitti_dir=/home/waterman/dataset/KITTI/ \
--pred_file=../results/kitti/test_kitti.npy \
--depth_results=../results/kitti_depths

evaluating ../results/kitti/test_kitti.npy...
[Info] Saving depth resutls to: ../results/kitti_depths
[NUM TEST]: 697


/home/garin/Documents/depth/src/kitti_eval/eval_depth.py:127: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  misc.imsave("%s/%s_pred_depth.png" % (depth_path, filename), colored_map)


   abs_rel,     sq_rel,        rms,    log_rms,         a1,         a2,         a3
    0.1591,     1.3648,     5.8362,     0.2329,     0.7923,     0.9313,     0.9729


#### 將結果傳回本地端 

scp -r -P 2222 garin@140.113.214.40:/home/garin/Documents/depth/results/kitti_depths/ C:/Users/Garin/Desktop/學長畢業資料/實驗程式yu/result

## TensorBoard 

In [ ]:
! tensorboard --logdir=../results/KITTI_RAW_256_832_UnDepthflow_dp_b4_ShuffleNetV2_3frames/ \
--samples_per_plugin images=100